# Excess chemical potential (Debye-Hückel theory)
Total molality:  
$m* = \sum\limits_i^{species} {{m_i}}  = \Gamma \frac{{\sum\limits_i^{species} {{X_i}} }}{{{X_w}}} = \Gamma \frac{{\sum\limits_i^{species} {{n_i}} }}{{{n_w}}}$  
Correction for mole fraction to molality conversion (introduced by Helgeson):  
${\Gamma _G} =  - RT\ln \left( {1 + \frac{{m*}}{\Gamma }} \right)$  
True inoic strength (note sum over species):  
$ \bar I = \frac{1}{2}\sum\limits_i^{species} {{m_i}z_i^2} $  
Denominator of Debye-Hückel term:  
$\Lambda  = 1 + {a_0}{B_\gamma }\sqrt {\bar I}$  
Excess chemical potential for ions and neutral species:  
$RT\ln {\gamma _i} = \frac{{z_i^2{A_G}\sqrt {\bar I} }}{{2\Lambda }} + {\Gamma _G}$  
Excess chemical potential for solvent (water):  
$\sigma  = 3\frac{{\Lambda  - \frac{1}{\Lambda } - 2\ln \left( \Lambda  \right)}}{{{{\left( {\Lambda  - 1} \right)}^3}}}$  
$RT\ln {\gamma _{{{\rm{H}}_{\rm{2}}}{\rm{O}}}} =  - \frac{{{A_G}{{\left( {\sqrt {\bar I} } \right)}^3}\sigma }}{{3\Gamma }} + {\Gamma _G}$  
### Total chemical potential
Water:  
$\mu _{{{\rm{H}}_{\rm{2}}}{\rm{O}}}^{} = {n_{{{\rm{H}}_{\rm{2}}}{\rm{O}}}}RT\ln {\gamma _{{{\rm{H}}_{\rm{2}}}{\rm{O}}}}$  
ions and neutral species:  
$\mu _i^{} = {n_i}RT\ln {m_i} + {n_i}RT\ln {\gamma _i}$

In [ ]:
import sympy as sym
from sympy.printing.ccode import C99CodePrinter
sym.init_printing()

In [ ]:
nb = 5
ns = 10
ne = nb + ns
i, R,T,P, CapGamG = sym.symbols('i R T P Gamma_G')

In [ ]:
b_list = []
b_T = sym.S.Zero
for j in range(0,nb):
    entry = sym.symbols('b_'+str(j))
    b_list.append(entry)
    b_T += entry
b = b_list
b, b_T

In [ ]:
class s(sym.Function):
    nargs = nb+3

In [ ]:
X = s(i, T, P, *b)/b_T
nW = b[0]
CapGam = sym.symbols('Gamma')
Xw = nW/b_T
m = X*CapGam/Xw
m

In [ ]:
class msum(sym.Function):
    nargs = nb
mstar = msum(*b)
mstar

In [ ]:
class z(sym.Function):
    nargs = 1

In [ ]:
Ibar = m*z(i)**2/2
Ibar

In [ ]:
CapGamG = -R*T*sym.log(1+mstar/CapGam)
CapGamG

In [ ]:
class aZero(sym.Function):
    nargs = nb
a0 = aZero(*b)
a0

In [ ]:
from thermoengine import coder
Bgamma = coder.Bgamma(T,P)
AsubG = coder.AsubG(T,P)

In [ ]:
CapLambda = 1 + a0*Bgamma*sym.sqrt(Ibar)
CapLambda

In [ ]:
RTlnGamma = z(i)**2*AsubG*sym.sqrt(Ibar)/2/CapLambda + CapGamG
RTlnGamma

$\sigma  = 3\frac{{\Lambda  - \frac{1}{\Lambda } - 2\ln \left( \Lambda  \right)}}{{{{\left( {\Lambda  - 1} \right)}^3}}}$  
$RT\ln {\gamma _{{{\rm{H}}_{\rm{2}}}{\rm{O}}}} =  - \frac{{{A_G}{{\left( {\sqrt {\bar I} } \right)}^3}\sigma }}{{3\Gamma }} + {\Gamma _G}$  

In [ ]:
RTlnGammaWater = -AsubG*(sym.sqrt(Ibar))**3*(CapLambda-1/CapLambda-2*sym.log(CapLambda))/CapGam/(CapLambda-1)**3 + CapGamG
RTlnGammaWater

In [ ]:
class SubCodePrinter(C99CodePrinter):
    def _print_s(self, expr):
        return str(expr.func) + '(' + str(expr.args[0]) + ', ' + str(expr.args[1]) + ', ' + str(expr.args[2]) + ', b)'
    def _print_z(self, expr):
        return str(expr.func) + '(' + str(expr.args[0]) + ')'
    def _print_KroneckerDelta(self, expr):
        i = expr.args[0]
        j = expr.args[1]
        return '((i==j) ? 1 : 0)'
    def _print_msum(self, expr):
        return str(expr.func) + '(b)'
    def _print_aZero(self, expr):
        return str(expr.func) + '(b)'
    def _print_Derivative(self, expr):
        function, *vars = expr.args
        number_of_derivatives = len(expr.args) - 1
        if function.func.__name__[0:1] == 's':
            if number_of_derivatives == 1:
                d_str = ''.join(repr(vars[0][0]).split('_'))
                d_ord  = '' if vars[0][1] == 1 else str(vars[0][1])
                result  = 'd' + d_ord + 's'
                for i in range(0,vars[0][1]):
                    result += 'D' + d_str
                result += '(' + str(function.args[0]) + ', ' + str(function.args[1]) + ', ' + str(function.args[2]) + ', b)'
            elif number_of_derivatives == 2:
                d_str1 = ''.join(repr(vars[0][0]).split('_'))
                d_str2 = ''.join(repr(vars[1][0]).split('_'))
                d_ord  = str(vars[0][1]+vars[1][1])
                result  = 'd' + d_ord + 's'
                for i in range(0,vars[0][1]):
                    result += 'D' + d_str1
                for i in range(0,vars[1][1]):
                    result += 'D' + d_str2
                result += '(' + str(function.args[0]) + ', ' + str(function.args[1]) + ', ' + str(function.args[2]) + ', b)'
            elif number_of_derivatives == 3:
                d_str1 = ''.join(repr(vars[0][0]).split('_'))
                d_str2 = ''.join(repr(vars[1][0]).split('_'))
                d_str3 = ''.join(repr(vars[2][0]).split('_'))
                d_ord  = str(vars[0][1]+vars[1][1]+vars[2][1])
                result  = 'd' + d_ord + 's'
                for i in range(0,vars[0][1]):
                    result += 'D' + d_str1
                for i in range(0,vars[1][1]):
                    result += 'D' + d_str2
                for i in range(0,vars[2][1]):
                    result += 'D' + d_str3
                result += '(' + str(function.args[0]) + ', ' + str(function.args[1]) + ', ' + str(function.args[2]) + ', b)'
            else:
                print ('Error: More than three derivatives of s not supported.')
                result = ''
        elif function.func.__name__[0:5] == 'aZero':
            if number_of_derivatives == 1:
                d_str1 = repr(vars[0][0]).split('_')[0]
                d_str2 = repr(vars[0][0]).split('_')[1]
                d_ord  = '' if vars[0][1] == 1 else str(vars[0][1])
                result  = 'd' + d_ord + 'aZero' + 'D' + d_str1 + d_ord
                for i in range(0,vars[0][1]):
                    result += '[' + d_str2 + ']'
            elif number_of_derivatives == 2:
                d_str11 = repr(vars[0][0]).split('_')[0]
                d_str12 = repr(vars[0][0]).split('_')[1]
                d_str21 = repr(vars[1][0]).split('_')[0]
                d_str22 = repr(vars[1][0]).split('_')[1]
                d_ord  = str(vars[0][1]+vars[1][1])
                assert d_str11 == d_str21, 'Error in derivative specification to aZero()'
                result  = 'd' + d_ord + 'aZero' + 'D' + d_str11 + d_ord
                for i in range(0,vars[0][1]):
                    result += '[' + d_str12 +']'
                for i in range(0,vars[1][1]):
                    result += '[' + d_str22 + ']'
            elif number_of_derivatives == 3:
                d_str11 = repr(vars[0][0]).split('_').split('_')[0]
                d_str12 = repr(vars[0][0]).split('_').split('_')[1]
                d_str21 = repr(vars[1][0]).split('_').split('_')[0]
                d_str22 = repr(vars[1][0]).split('_').split('_')[1]
                d_str31 = repr(vars[2][0]).split('_').split('_')[0]
                d_str32 = repr(vars[2][0]).split('_').split('_')[1]
                d_ord  = str(vars[0][1]+vars[1][1]+vars[2][1])
                assert d_str11 == d_str21, 'Error in derivative specification to aZero()'
                assert d_str21 == d_str31, 'Error in derivative specification to aZero()'
                result  = 'd' + d_ord + 'aZero' + 'D' + d_str11 + d_ord
                for i in range(0,vars[0][1]):
                    result += '[' + d_str12 + ']'
                for i in range(0,vars[1][1]):
                    result += '[' + d_str22 + ']'
                for i in range(0,vars[2][1]):
                    result += '[' + d_str32 + ']'
            else:
                print ('Error: More than three derivatives of aZero not supported.')
                result = ''
        elif function.func.__name__[0:4] == 'msum':
            if number_of_derivatives == 1:
                d_str1 = repr(vars[0][0]).split('_')[0]
                d_str2 = repr(vars[0][0]).split('_')[1]
                d_ord  = '' if vars[0][1] == 1 else str(vars[0][1])
                result  = 'd' + d_ord + 'msum' + 'D' + d_str1 + d_ord
                for i in range(0,vars[0][1]):
                    result += '[' + d_str2 + ']'
            elif number_of_derivatives == 2:
                d_str11 = repr(vars[0][0]).split('_')[0]
                d_str12 = repr(vars[0][0]).split('_')[1]
                d_str21 = repr(vars[1][0]).split('_')[0]
                d_str22 = repr(vars[1][0]).split('_')[1]
                d_ord  = str(vars[0][1]+vars[1][1])
                assert d_str11 == d_str21, 'Error in derivative specification to msum()'
                result  = 'd' + d_ord + 'msum' + 'D' + d_str11 + d_ord
                for i in range(0,vars[0][1]):
                    result += '[' + d_str12 +']'
                for i in range(0,vars[1][1]):
                    result += '[' + d_str22 + ']'
            elif number_of_derivatives == 3:
                d_str11 = repr(vars[0][0]).split('_').split('_')[0]
                d_str12 = repr(vars[0][0]).split('_').split('_')[1]
                d_str21 = repr(vars[1][0]).split('_').split('_')[0]
                d_str22 = repr(vars[1][0]).split('_').split('_')[1]
                d_str31 = repr(vars[2][0]).split('_').split('_')[0]
                d_str32 = repr(vars[2][0]).split('_').split('_')[1]
                d_ord  = str(vars[0][1]+vars[1][1]+vars[2][1])
                assert d_str11 == d_str21, 'Error in derivative specification to msum()'
                assert d_str21 == d_str31, 'Error in derivative specification to msum()'
                result  = 'd' + d_ord + 'msum' + 'D' + d_str11 + d_ord
                for i in range(0,vars[0][1]):
                    result += '[' + d_str12 + ']'
                for i in range(0,vars[1][1]):
                    result += '[' + d_str22 + ']'
                for i in range(0,vars[2][1]):
                    result += '[' + d_str32 + ']'
            else:
                print ('Error: More than three derivatives of msum not supported.')
                result = ''
        elif (len(function.func.__name__) >= 6 and 
            function.func.__name__[1:6] == 'gamma'):
            if number_of_derivatives == 1:
                derivative_string = repr(vars[0][0]).lower()
                derivative_order  = '' if vars[0][1] == 1 else str(vars[0][1])
                result = ('d' + derivative_order + function.func.__name__ + 'd' 
                    + derivative_string + derivative_order + '(T, P)')
            elif number_of_derivatives == 2:
                derivative_string_2 = repr(vars[0][0]).lower()
                derivative_order_2  = '' if vars[0][1] == 1 else str(vars[0][1])
                derivative_string_1 = repr(vars[1][0]).lower()
                derivative_order_1  = '' if vars[1][1] == 1 else str(vars[1][1])
                derivative_total    = str(vars[0][1]+vars[1][1])
                result = ('d' + derivative_total + function.func.__name__ + 'D' 
                    + derivative_string_1 + derivative_order_1 +'D' 
                    + derivative_string_2 + derivative_order_2 + '(T, P)')
            else:
                result = ''
        else:
            if (not isinstance(type(function), UndefinedFunction) or 
                not all(isinstance(i, Symbol) for i in vars)):
                return super()._print_Derivative(expr)
        return result
function_d = {"Debye":"Debye", "B":"born_B", "Y":"born_Y", "Q":"born_Q", 
              "X":"born_X", "U":"born_U", "N":"born_N", "dXdT":"born_dXdT", 
              "dUdT":"born_dUdT", "dNdT":"born_dNdT", "dUdP":"born_dUdP", 
              "dNdP":"born_dNdP", "gSolvent":"gSolvent", 
              "Agamma":"Agamma", "Bgamma":"Bgamma", "AsubG":"AsubG", 
              "AsubH":"AsubH", "AsubV":"AsubV", "AsubJ":"AsubJ", 
              "AsubKappa":"AsubKappa", "AsubEx":"AsubEx", 
              "BsubG":"BsubG", "BsubH":"BsubH", "BsubV":"BsubV", 
              "BsubJ":"BsubJ", "BsubKappa":"BsubKappa", "BsubEx":"BsubEx"}
printer = SubCodePrinter(settings={"user_functions":function_d})

In [ ]:
def substitute(in_s):
    sub_d = { 'b_0':'b[0]'}
    for key,value in sub_d.items():
        out_s = in_s.replace(key, value)
        in_s = out_s
    return out_s

In [ ]:
print(substitute(printer.doprint(RTlnGamma, assign_to='RTlngamma[i]')))

In [ ]:
print(substitute(printer.doprint(RTlnGammaWater, assign_to='RTlngammaH2O')))

In [ ]:
print(substitute(printer.doprint(RTlnGamma.diff(T), assign_to='dRTlngammaDt[i]')))

In [ ]:
print(substitute(printer.doprint(RTlnGamma.diff(T,2), assign_to='d2RTlngammaDt2[i]')))

In [ ]:
print(substitute(printer.doprint(RTlnGamma.diff(T,P), assign_to='d2RTlngammaDtDp[i]')))

In [ ]:
print(substitute(printer.doprint(RTlnGamma.diff(P), assign_to='dRTlngammaDp[i]')))

In [ ]:
print(substitute(printer.doprint(RTlnGamma.diff(P,2), assign_to='d2RTlngammaDp2[i]')))

In [ ]:
print(substitute(printer.doprint(RTlnGamma.diff(b[0]), assign_to='dRTlngammaDb0[i]')))

In [ ]:
print(substitute(printer.doprint(RTlnGamma.diff(b[1]), assign_to='dRTlngammaDb1[i]')))

In [ ]:
print(substitute(printer.doprint(RTlnGamma.diff(b[0],2), assign_to='d2RTlngammaDb0Db0[i]')))

In [ ]:
print(substitute(printer.doprint(RTlnGamma.diff(b[0],b[1]), assign_to='d2RTlngammaDb0Db1[i]')))

In [ ]:
print(substitute(printer.doprint(RTlnGamma.diff(b[0],3), assign_to='d2RTlngammaDb0Db0Db0[i]')))